In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as panda

class Principal(webdriver.Chrome):

    def __init__(self):
        options = webdriver.ChromeOptions().add_experimental_option('excludeSwitches', ['enable-loggin'])
        service = Service(ChromeDriverManager().install())
        super(Principal, self).__init__(service = service, options = options)
        self.get("https://voluntarias.plataformamaisbrasil.gov.br/voluntarias/Principal/Principal.do?Usr=guest&Pwd=guest")

    def plataformaMaisBrasil(self):
        self.implicitly_wait(5)

        column = self.find_element(By.CLASS_NAME, "col1")

        convenio = column.find_elements(By.TAG_NAME, "div")[3]

        convenio.click()
        consultaConvenios = self.find_element(By.TAG_NAME, "li")
        consultaConvenios.click()
    def consultarConvenio(self):
        cnpj = panda.read_excel("./solicitacoes.xlsx", index_col=0)
        numero_convenios = cnpj.index.values
        numero_convenios_encontrados = []
        cnpj_encontrados = []

        for numero_convenio in numero_convenios:
            self.get()
            input_convenio = self.find_element(By.ID, "consultarNumeroConvenio")
            form_submit = self.find_element(By.ID, "form_submit")
            input_convenio.send_keys(str(numero_convenio))
            form_submit.click()
            table_line = self.find_element(By.ID, "listaResultado")
            if(table_line.text == "Nenhum registro foi encontrado."):
                self.plataformaMaisBrasil()
            else:
                table_line.find_elements(By.TAG_NAME, "a")[0].click()
                proponente = self.find_element(By.CLASS_NAME, "proponente")
                cnpj = proponente.find_elements(By.TAG_NAME, "td")[1].text.split(" ")[1]
                numero_convenios_encontrados.append(numero_convenio)
                cnpj_encontrados.append(cnpj)
                self.plataformaMaisBrasil()
            
            
        dados = {'numero': numero_convenios_encontrados, 'cnpj': cnpj_encontrados}
        data_frame_resposta = panda.DataFrame(data = dados)
        data_frame_resposta.to_excel("resposta.xlsx", index=False)    
        
bot = Principal()


bot.plataformaMaisBrasil()
bot.consultarConvenio()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()